<a href="https://colab.research.google.com/github/asifahsaan/NLP-Natural-Language-Processing-/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**SENTIMENT ANALYSIS** </center>

<br>
<center>

|Title:     | Tweets into Multiclass classification|
|         :-            |                             :-:                               |
| **Resource Person:**  | Dr. Maryam                                      |
| **Course Title:**     | Natural Language Processing                                                  |
| **Credit Hours:**     | 3(3,0)                                                        |
| **Work Objective:**     | Learning of Text data and ML Classification Model                                                   |
<br>
            |          **Submitted By** |

|Sr. # | Student Name                   | Registration No. | Email Address           |
| :-   | :-                     | :-               | :-                      
| 1    | Asif Ahsan | 20L-2370         | l202370@lhr.nu.edu.pk |

</center>
<br>

### **Importing Libraries**

In [ ]:
!pip3 install snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")

import re
import nltk
nltk.download('punkt')
import string
string.punctuation
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

try:
  %load_ext google.colab.data_table
except:
  %reload_ext google.colab.data_table

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 2.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### **(A). Prepare a Roman Urdu dataset for multiclass classification (sentiment analysis)**

**Scrapping tweets and stroing them into csv.**

In [ ]:
# Define the query to search for tweets
query = "lockdown lag gaya lang:en"

# Define the number of tweets to extract
num_tweets = 500

# Define a list to store the tweets
tweets = []

# Loop through the tweets and extract the text content
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i >= num_tweets:
        break
    tweets.append(tweet.content)

# Create a pandas DataFrame to store the tweets
df = pd.DataFrame({'tweet': tweets})
# Save the DataFrame to a CSV file
df.to_csv('tweets.csv', index=False)
print("Data scrapped and stored successfully....")

Data scrapped and stored successfully....


In [ ]:
df

,date,tweet Id,text
0,2023-02-19 05:01:31+00:00,1627171497604444160,@aamir_khan_fan2 srk ki pathaan bolckbuster ho...
1,2023-02-11 03:26:18+00:00,1624248435418484736,I was gay for him yeah !! Anyways keeping it a...
2,2022-12-26 16:12:52+00:00,1607409117307752448,@mddubey409 @on_drive23 I'm going home tomorro...
3,2022-08-17 17:32:09+00:00,1559956257548615680,@Middlclssmowgli @_touchgrasss 3.. last time l...
4,2022-08-17 09:54:01+00:00,1559840965954863106,@nikiru17 10th class tak hota tha fir lockdown...
...,...,...,...
439,2020-03-23 04:11:40+00:00,1241940654084644867,Due to lockdown everywhere.\nSuddenly aisa nah...
440,2020-03-22 14:30:39+00:00,1241734036126445569,@ItsAnup_ Bhai hone lag gaya h wo bhi.\nKarnat...
441,2020-03-22 11:23:46+00:00,1241687005144027136,Section 144 lag toh gaya hai. Aur kya chahiye?...
442,2020-03-21 08:31:23+00:00,1241281238901690368,"I am so so fed up with people who say ,Allah k..."


**Assigning class to scarpped tweets**

In [ ]:
from textblob import TextBlob

# Load the tweets from the CSV file into a pandas DataFrame
df = pd.read_csv('tweets.csv', encoding="utf-8")

# Define a function to get the sentiment label for a tweet
def get_sentiment_label(tweet):
    # Use TextBlob to get the polarity of the tweet
    polarity = TextBlob(tweet).sentiment.polarity
    
    # Assign a sentiment label based on the polarity
    if polarity < -0.5:
        return 'very negative'
    elif polarity < 0:
        return 'negative'
    elif polarity == 0:
        return 'neutral'
    elif polarity <= 0.5:
        return 'positive'
    else:
        return 'very positive'

# Apply the get_sentiment_label function to the tweets DataFrame to create a new column with the sentiment labels
df['sentiment'] = df['text'].apply(get_sentiment_label)

# Save the labeled DataFrame to a CSV file
df.to_csv('tweets_labeled.csv', index=False, encoding="utf-8")
df.head()

,date,tweet Id,text,sentiment
0,2023-02-19 05:01:31+00:00,1627171497604444160,@aamir_khan_fan2 srk ki pathaan bolckbuster ho...,negative
1,2023-02-11 03:26:18+00:00,1624248435418484736,I was gay for him yeah !! Anyways keeping it a...,very positive
2,2022-12-26 16:12:52+00:00,1607409117307752448,@mddubey409 @on_drive23 I'm going home tomorro...,negative
3,2022-08-17 17:32:09+00:00,1559956257548615680,@Middlclssmowgli @_touchgrasss 3.. last time l...,neutral
4,2022-08-17 09:54:01+00:00,1559840965954863106,@nikiru17 10th class tak hota tha fir lockdown...,neutral


In [ ]:
print("Tweets classified into")
print(df.sentiment.unique())
print()
print("Classes distribution count")
print(df.sentiment.value_counts())

Tweets classified into
['negative' 'very positive' 'neutral' 'positive' 'very negative']

Classes distribution count
neutral          206
positive         136
negative          76
very positive     13
very negative     13
Name: sentiment, dtype: int64


As we can check that we have classified our tweets into 5 different classes ['negative', 'very positive', 'neutral', 'positive',
       'very negative'] as required. Now we will do some preprocessing and label encoding before building our classifiers

#### **Data Preprocessing**

**Cleaning our data**

In [ ]:
def clean_data(text):
  text_to_clean=str(text).lower()
  text_to_clean2 = re.sub("@[A-Za-z0-9_]+"," ", text_to_clean ) #remove anything starting with handler @ e.g remove @abc
  lowercase_text = re.sub("#[A-Za-z0-9_]+"," ", text_to_clean2 ) #remove hashtags words
  lowercase_text= re.sub('https?://\S+|www\.\S+', '',   lowercase_text ) # remove urls
  lowercase_text=re.sub("[^a-zA-Z\s]"," ",lowercase_text)  
  lowercase_text=re.sub(r'\brt\b', '', lowercase_text) #remove rt word 
  exclude = set(string.punctuation)   
  text = ''.join(ch for ch in lowercase_text if ch not in exclude)   
  text=re.sub('[“”’\']', "", str(text)) #remove punctuation inverted comma apostrophe
  text = re.sub('[0-9]', "", str(text)) #remove digits
  text = re.sub('\n', "", str(text))  
  cleantext=re.sub("\s{2,}",' ',str(text)) #remove spaces more than 2 
  cleantext=cleantext.strip() 
  return cleantext

In [ ]:
df["CleanedTweets"] = df["text"].apply(lambda x:clean_data(x))
df.head()

,date,tweet Id,text,sentiment,CleanedTweets
0,2023-02-19 05:01:31+00:00,1627171497604444160,@aamir_khan_fan2 srk ki pathaan bolckbuster ho...,negative,srk ki pathaan bolckbuster hogyi toh sab super...
1,2023-02-11 03:26:18+00:00,1624248435418484736,I was gay for him yeah !! Anyways keeping it a...,very positive,i was gay for him yeah anyways keeping it asid...
2,2022-12-26 16:12:52+00:00,1607409117307752448,@mddubey409 @on_drive23 I'm going home tomorro...,negative,i m going home tomorrow and agar lockdown lag ...
3,2022-08-17 17:32:09+00:00,1559956257548615680,@Middlclssmowgli @_touchgrasss 3.. last time l...,neutral,last time lockdown lag gaya tha
4,2022-08-17 09:54:01+00:00,1559840965954863106,@nikiru17 10th class tak hota tha fir lockdown...,neutral,th class tak hota tha fir lockdown lag gaya


In [ ]:
df['CleanedTweets'].to_csv("clearText.csv")

As we have now our data cleaned completely and we are done with preprocessing so now we will move towards our final goal which is calssification of tweets.

## **(B). Classification of tweets**
We will be doing  classification as:


1.   ***Multiclass Classification***

We will implement these classifiers for our classifications: <br>
1.   *Naive Bayes*
2.   *Logistic Regression*
3.   *Random Forest*
4.   *SVM*
5.   *Perceptron*

**Split dataset into Train/Test and define classifiers**

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['CleanedTweets'], df['sentiment'], 
                                                    test_size=0.2, random_state=42)

# Define the classifiers
nb = MultinomialNB()
lr = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(n_estimators=100)
svm = SVC(kernel='linear')
perc = Perceptron()

### **2.1. BOW Raw CountVectorizer Model**

In [ ]:
# Define the vectorizers for the bag of words raw counts
cv = CountVectorizer()

# Vectorize the training and testing data using the bag of words model
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [ ]:
from sklearn.metrics import classification_report
import time
start = time.time()

# Train and evaluate the classifiers using the bag of words model
count=0
for clf in [nb, lr, rf, svm, perc]:
    clf.fit(X_train_cv, y_train)
    y_pred = clf.predict(X_test_cv)
    print(count+1,":", type(clf).__name__)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print("- MICRO AVERAGING")
    print('Precision:', precision_score(y_test, y_pred, average='micro'))
    print('Recall:', recall_score(y_test, y_pred, average='micro'))
    print('F-score:', f1_score(y_test, y_pred, average='micro'))
    print("- MACRO AVERAGING")
    print('Precision:', precision_score(y_test, y_pred, average='macro'))
    print('Recall:', recall_score(y_test, y_pred, average='macro'))
    print('F-score:', f1_score(y_test, y_pred, average='macro'))
    print("Classification Report")
    print(classification_report(y_test,y_pred))
    print()
    stop = time.time()
    print("\t\tTime taken by model to get train =", stop - start,"seconds\n")
    count = count+1

1 : MultinomialNB
Accuracy: 0.48314606741573035
- MICRO AVERAGING
Precision: 0.48314606741573035
Recall: 0.48314606741573035
F-score: 0.48314606741573035
- MACRO AVERAGING
Precision: 0.19844444444444445
Recall: 0.2664664664664665
F-score: 0.2274767378215654
Classification Report
               precision    recall  f1-score   support

     negative       0.00      0.00      0.00        20
      neutral       0.52      0.70      0.60        37
     positive       0.47      0.63      0.54        27
very negative       0.00      0.00      0.00         2
very positive       0.00      0.00      0.00         3

     accuracy                           0.48        89
    macro avg       0.20      0.27      0.23        89
 weighted avg       0.36      0.48      0.41        89


		Time taken by model to get train = 0.03055572509765625 seconds

2 : LogisticRegression
Accuracy: 0.5168539325842697
- MICRO AVERAGING
Precision: 0.5168539325842697
Recall: 0.5168539325842697
F-score: 0.5168539325842697


### **2.2. BOW TFIDF vectorizer Model**

In [ ]:
# Define the vectorizers for the bag of words TFIDFVectorizer
tv = TfidfVectorizer(stop_words='english', use_idf=True, max_features = 5000)

# Vectorize the training and testing data using the n-gram model
X_train_tv = tv.fit_transform(X_train)
X_test_tv = tv.transform(X_test)

In [ ]:
import time
start = time.time()

# Train and evaluate the classifiers using the TfIDF model
count=0
for clf in [nb, lr, rf, svm, perc]:
    clf.fit(X_train_tv, y_train)
    y_pred = clf.predict(X_test_tv)
    print(count+1,":", type(clf).__name__)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print("- MICRO AVERAGING")
    print('Precision:', precision_score(y_test, y_pred, average='micro'))
    print('Recall:', recall_score(y_test, y_pred, average='micro'))
    print('F-score:', f1_score(y_test, y_pred, average='micro'))
    print("- MACRO AVERAGING")
    print('Precision:', precision_score(y_test, y_pred, average='macro'))
    print('Recall:', recall_score(y_test, y_pred, average='macro'))
    print('F-score:', f1_score(y_test, y_pred, average='macro'))
    print("Classification Report")
    print(classification_report(y_test,y_pred))
    print()
    stop = time.time()
    print("\t\tTime taken by model to get train =", stop - start,"seconds\n")
    count = count+1

1 : MultinomialNB
Accuracy: 0.449438202247191
- MICRO AVERAGING
Precision: 0.449438202247191
Recall: 0.449438202247191
F-score: 0.449438202247191
- MACRO AVERAGING
Precision: 0.23705882352941177
Recall: 0.22222222222222224
F-score: 0.16002115282919088
Classification Report
               precision    recall  f1-score   support

     negative       0.00      0.00      0.00        20
      neutral       0.44      1.00      0.61        37
     positive       0.75      0.11      0.19        27
very negative       0.00      0.00      0.00         2
very positive       0.00      0.00      0.00         3

     accuracy                           0.45        89
    macro avg       0.24      0.22      0.16        89
 weighted avg       0.41      0.45      0.31        89


		Time taken by model to get train = 0.03286099433898926 seconds

2 : LogisticRegression
Accuracy: 0.47191011235955055
- MICRO AVERAGING
Precision: 0.47191011235955055
Recall: 0.47191011235955055
F-score: 0.47191011235955055
- 

### **2.3. NGrams (UniGram, BiGram, TriGram) Model**

In [ ]:
# Define the vectorizers for the bag of words N-Grams
ngram_cv = CountVectorizer(analyzer='char_wb', ngram_range=(1, 3))

# Vectorize the training and testing data using the N-Grams model
X_train_nGramCV = ngram_cv.fit_transform(X_train)
X_test_nGramCV = ngram_cv.transform(X_test)

In [ ]:
import time
start = time.time()

# Train and evaluate the classifiers using the n-gram model
count=0
for clf in [nb, lr, rf, svm, perc]:
    clf.fit(X_train_nGramCV, y_train)
    y_pred = clf.predict(X_test_nGramCV)
    print(count+1,":", type(clf).__name__)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print("- MICRO AVERAGING")
    print('Precision:', precision_score(y_test, y_pred, average='micro'))
    print('Recall:', recall_score(y_test, y_pred, average='micro'))
    print('F-score:', f1_score(y_test, y_pred, average='micro'))
    print("- MACRO AVERAGING")
    print('Precision:', precision_score(y_test, y_pred, average='macro'))
    print('Recall:', recall_score(y_test, y_pred, average='macro'))
    print('F-score:', f1_score(y_test, y_pred, average='macro'))
    print("Classification Report")
    print(classification_report(y_test,y_pred))
    print()
    stop = time.time()
    print("\t\tTime taken by model to get train =", stop - start,"seconds\n")
    count = count+1

1 : MultinomialNB
Accuracy: 0.449438202247191
- MICRO AVERAGING
Precision: 0.449438202247191
Recall: 0.449438202247191
F-score: 0.449438202247191
- MACRO AVERAGING
Precision: 0.2296153019557275
Recall: 0.25143143143143143
F-score: 0.23330049261083743
Classification Report
               precision    recall  f1-score   support

     negative       0.22      0.10      0.14        20
      neutral       0.53      0.68      0.60        37
     positive       0.39      0.48      0.43        27
very negative       0.00      0.00      0.00         2
very positive       0.00      0.00      0.00         3

     accuracy                           0.45        89
    macro avg       0.23      0.25      0.23        89
 weighted avg       0.39      0.45      0.41        89


		Time taken by model to get train = 0.04267621040344238 seconds

2 : LogisticRegression
Accuracy: 0.4157303370786517
- MICRO AVERAGING
Precision: 0.4157303370786517
Recall: 0.4157303370786517
F-score: 0.4157303370786517
- MACRO